### Merge each raw xlsx file to mergeRaw.csv
xlsx file에 대해 불필요한 열을 제거하고 종영일 column을 추가하여
각 주차에 대한 박스오피스 데이터를 단순한 하나의 csv로 병합하는 코드입니다

kobis 데이터를 받았을 때 xls 형식으로 다운로드 되는데, 
해당 확장자의 파일을 바로 열게 되면 오류가 발생하여 xlsx로 변환하는 과정이 필요합니다.

그래서 만약 1년 이상의 추가적인 박스 오피스 데이터를 추가하게 되어서 이 코드가 다시 필요해진다면
convertToXlsx.ipynb를 먼저 실행해서 확장자를 xlsx로 먼저 변환한 후에, 이 파일을 실행시켜주시면 되겠습니다.

In [1]:
# 파일명을 input으로 받아 
# 조회 일자 date column을 추가하는 함수
import os
import pandas as pd
import re

def load_each_files(file_name: str, base_dir: str = None) -> pd.DataFrame:
    # 시작 디렉토리 결정
    if base_dir is None:
        base_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # 파일 경로 조합
    raw_path = os.path.join(base_dir, "data", "raw(xlsx)", file_name)

    # (1) 파일 읽기
    df = pd.read_excel(raw_path, engine="openpyxl")

    # (2) 첫 번째 컬럼 이름
    first_col = df.columns[0]

    # (3) “●YYYY년 MM월 DD일(…)” 패턴 식별
    date_mask = df[first_col].astype(str).str.contains(r'^\s*●\s*\d{4}년', na=False)

    # (4) raw_date 추출
    df['raw_date'] = (
        df[first_col]
        .where(date_mask)
        .str.extract(r'●\s*([\d]{4}년\s*\d{1,2}월\s*\d{1,2}일)')[0]
    )

    # (5) datetime 변환
    df['date'] = (
        df['raw_date']
          .str.replace(r'년\s*', '-', regex=True)
          .str.replace(r'월\s*', '-', regex=True)
          .str.replace(r'일',   '',  regex=True)
          .pipe(pd.to_datetime, format='%Y-%m-%d')
    )

    # (6) 날짜 FFILL
    df['date'] = df['date'].ffill()

    # (7) 불필요 행 제거 (날짜, “순위” 헤더 등)
    header_mask = df[first_col].astype(str).str.contains(r'순위', na=False)
    df_clean = df.loc[~(date_mask | header_mask)].copy().reset_index(drop=True)

    # (8) raw_date 컬럼 삭제
    df_clean = df_clean.drop(columns=['raw_date'])

    return df_clean


In [2]:
# 유효한 행만 필터링 하는 함수
import pandas as pd

def filter_numeric_rows(df: pd.DataFrame, col: str | int = 0) -> pd.DataFrame:
    filtered_rows = []
    # 컬럼 색인 정리
    if isinstance(col, int):
        col_name = df.columns[col]
    else:
        col_name = col

    for _, row in df.iterrows():
        val = row.iloc[df.columns.get_loc(col_name)]
        # 1) NaN은 건너뛰기
        if pd.isna(val):
            continue
        # 2) 문자열로 바꾼 뒤 앞뒤 공백 제거
        s = str(val).strip()
        # 3) 온전히 숫자로만 이루어진 경우에만 저장
        if s.isdigit():
            filtered_rows.append(row)

    # 원본 컬럼 구조 유지하며 DataFrame 생성
    return pd.DataFrame(filtered_rows, columns=df.columns)


In [3]:
# 모든 xlsx 파일에 대해 garbage row를 제거하여 하나의 csv로 병합하는 코드

import os
import pandas as pd

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
xlsx_dir = os.path.join(BASE_DIR, "data", "raw(xlsx)")

all_filtered = []

for root, dirs, files in os.walk(xlsx_dir):
    for fname in files:
        if fname.lower().endswith(".xlsx"):
            file_path = os.path.join(root, fname)
            
            df_clean = load_each_files(file_path)
            df_filtered = filter_numeric_rows(df_clean)
            
            all_filtered.append(df_filtered)

if all_filtered:
    filtered_df = pd.concat(all_filtered, ignore_index=True)
else:
    filtered_df = pd.DataFrame(columns=df_clean.columns)


In [4]:
# date 오름차순 정렬 후 인덱스 재설정
filtered_df_sorted = filtered_df.sort_values(by='date').reset_index(drop=True)

In [9]:
col_str = """순위	영화명	개봉일	매출액 	매출액(점유율)	매출액증감	매출액증감율(전일대비) 	누적매출액 	관객수 	관객수증감(전일대비) 	관객수증감율(전일대비) 	누적관객수 	스크린수 	상영횟수 	대표국적 	국적 	제작사 	배급사 	등급 	장르 	감독 	배우 	조회일"""
col_str = """순위	영화명	개봉일	매출액 	매출액(점유율)	매출액증감	매출액증감율(전일대비) 	누적매출액 	관객수 	관객수증감(전일대비) 	관객수증감율(전일대비) 	누적관객수 	스크린수 	상영횟수 	대표국적 	국적 	제작사 	배급사 	등급 	장르 	감독 	배우 	조회일"""
new_cols = [c.strip() for c in col_str.split('\t') if c.strip()]

# 새 컬럼명 할당
filtered_df_sorted.columns = new_cols

In [10]:
filtered_df_sorted.to_csv('../data/mergedRaw/mergedRaw.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 개별 xlsx에 대해 garbage row 제거 예시 (참고고)
BASE_DIR  = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_fname = os.path.join(BASE_DIR, "data", "preprocessed", "KOBIS_일별_박스오피스_2025-05-15.xlsx")

df_clean = load_each_files(raw_fname)
filtered_df = filter_numeric_rows(df_clean)

In [ ]:
filtered_df[15000:15200]

In [6]:
filtered_df_sorted

,순위,영화명,개봉일,매출액,매출액,매출액증감,매출액증감율(전일대비),누적매출액,관객수,관객수증감(전일대비),...,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우,종영일
0,11,날씨의 아이,2019-10-30,2791900,0.001,2791900,1.000,2121328590,338,338,...,3,일본,일본,NaN,"메가박스중앙(주) 플러스엠 엔터테인먼트,(주)미디어캐슬,워터홀컴퍼니(주)",15세이상관람가,"애니메이션,판타지,멜로/로맨스",신카이 마코토,"다이고 코타로,모리 나나,오구리 슌,심규혁,최한,강은애,김서영",2024-05-01
1,17,DMZ 동물 특공대,2024-02-14,819000,0.000,819000,1.000,34081000,117,117,...,1,한국,한국,자루 스튜디오,(주)박수엔터테인먼트,전체관람가,애니메이션,홍인표,"김새해,조경이,황창영,이소은,박준형",2024-05-01
2,15,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,2023-12-22,1510539,0.001,1510539,1.000,1471461912,166,166,...,9,일본,일본,NaN,(주)씨제이이엔엠,전체관람가,애니메이션,오네 히토시,NaN,2024-05-01
3,14,슈가│어거스트 디 투어 ‘디-데이’ 더 무비,2024-04-10,5100000,0.003,4225000,4.829,533621500,204,169,...,4,한국,한국,(주)하이브,씨지브이 아이스콘,12세이상관람가,공연,박준수,민윤기,2024-05-01
4,13,캐벌케이드,NaT,431000,0.000,87000,0.253,4540000,208,36,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17253,25,기동전사 건담 F91,2025-05-07,254100,0.001,-422000,-0.624,13284700,25,-37,...,2,일본,일본,NaN,(주)미디어캐슬,12세이상관람가,애니메이션,토미노 요시유키,NaN,2025-05-14
17254,23,예르미타시 예술의 힘,2021-05-12,487000,0.002,359000,2.805,12981800,37,24,...,2,이탈리아,이탈리아,NaN,"일미디어,씨지브이 아이스콘",전체관람가,다큐멘터리,NaN,토니 세르빌로,2025-05-14
17255,22,굿바이 마이 라이프,NaT,78000,0.000,16000,0.258,1008000,39,8,...,2,미국,미국,NaN,NaN,NaN,드라마,빅터 플레밍,"스펜서 트레이시,라이오넬 배리모어",2025-05-14
17256,20,위드 러브,2025-05-14,369900,0.002,-8100100,-0.956,8839900,43,-1167,...,21,미국,미국,NaN,유니버설픽쳐스인터내셔널 코리아(유),청소년관람불가,액션,조나단 유세비오,"키 호이 콴,아리아나 데보스,오언조",2025-05-14
